## セットアップ

↓こちらを元に実装しています

<https://medium.com/data-and-beyond/complete-guide-to-building-bert-model-from-sratch-3e6562228891>

In [ ]:
# !uv add  transformers datasets tokenizers
import os
import shutil

PROJECT_ROOT = os.path.realpath("../")
!echo $PROJECT_ROOT

!wget http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip

!unzip -qqo cornell_movie_dialogs_corpus.zip

if not os.path.exists(f"{PROJECT_ROOT}/notebook/cornell movie-dialogs corpus"):
    os.rmdir(f"{PROJECT_ROOT}/notebook/cornell movie-dialogs corpus")

!rm cornell_movie_dialogs_corpus.zip

if not os.path.exists(f"{PROJECT_ROOT}/datasets"):
    os.makedirs(f"{PROJECT_ROOT}/datasets")
    # ファイルの移動
    os.rename(f"{PROJECT_ROOT}/notebook/cornell movie-dialogs corpus/movie_conversations.txt", 
            f"{PROJECT_ROOT}/datasets/movie_conversations.txt")
    os.rename(f"{PROJECT_ROOT}/notebook/cornell movie-dialogs corpus/movie_lines.txt", 
            f"{PROJECT_ROOT}/datasets/movie_lines.txt")
    # 後片付け
    shutil.rmtree(f"{PROJECT_ROOT}/notebook/cornell movie-dialogs corpus")
    shutil.rmtree(f"{PROJECT_ROOT}/notebook/__MACOSX")

## ダウンロードしたデータセットの形式

今回は「Cornel Movie-Dialogs Corpus」を用います。

[こちらより](https://arc.net/l/quote/telfmlkx)：

> For our tutorial, we will be utilizing the Cornell Movie-Dialogs Corpus, a vast collection of over 220,000 conversational exchanges between more than 10,000 pairs of characters in various movies and TV shows.

DeepL先生
> このチュートリアルでは、コーネル・ムービー・ダイアログ・コーパスを利用します。コーネル・ムービー・ダイアログ・コーパスは、様々な映画やテレビ番組に登場する10,000組以上のキャラクター間の220,000以上の会話のやり取りを収録した膨大なコレクションです。

続いて、各データの形式を確認していきます。


### 1. movie_conversations.txt

データ形式
- 区切り文字`+++$+++`

登場人物ID 1, 登場人物ID 2, 映画ID, 一連の会話の会話ID?


In [54]:
!less $PROJECT_ROOT/datasets/movie_conversations.txt | head -n 10

u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L271', 'L272', 'L273', 'L274', 'L275']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L276', 'L277']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L280', 'L281']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L363', 'L364']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L365', 'L366']


### 2. movie_lines.txt
データ形式
行ID, 人物ID, 映画ID, 人物名, 会話

In [55]:
!less $PROJECT_ROOT/datasets/movie_lines.txt | head -n 10

L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!
L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!
L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.
L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?
L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.
L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow
L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.
L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No
L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I'm kidding.  You know how sometimes you just become this "persona"?  And you don't know how to quit?
L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?


In [56]:
# ↓これが一連の会話の流れ？
!grep -nE "L(194|195|196|197) " $PROJECT_ROOT/datasets/movie_lines.txt

66:L197 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?
67:L196 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Not the hacking and gagging and spitting part.  Please.
68:L195 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Well, I thought we'd start with pronunciation, if that's okay with you.
69:L194 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.


## データのロードの実装
`src/bert_tutorial_data_loader.py`


## トークナイザーの実装

[この章](https://arc.net/l/quote/nrfqdjbr)のメモ

- BERTのトークナイザーはWordPieceトークナイザーを使う
- WordPieceトークナイザー
  - 一つの単語を複数のトークンに分割する可能性がある
    - ex. surfboarding → ['surf', '##boarding', '##ing']
    - `##`は前の単語と繋がっていたことを示すマーカー
  - [Hugging Faceの解説](https://huggingface.co/learn/nlp-course/chapter6/6?fw=pt)
    - かしこい…!